### Download Flowers dataset

In [1]:
#You can download the data manually as well instead of using 'wget'
!wget http://download.tensorflow.org/example_images/flower_photos.tgz --quiet

In [2]:
!tar -xf flower_photos.tgz

In [3]:
!ls -l

total 223464
drwxr-x--- 7 270850 5000      4096 Feb 10  2016 flower_photos
-rw-r--r-- 1 root   root 228813984 Feb 10  2016 flower_photos.tgz
drwxr-xr-x 1 root   root      4096 Dec 21 17:29 sample_data


In [4]:
#Read the dataset
import tarfile
dataset = tarfile.open('flower_photos.tgz')

In [5]:
#We will build a pandas dataset
import pandas as pd
df = pd.DataFrame(columns=['class','image_file'])

In [6]:
#Run through tarfile members 
for name in dataset.getnames():
    
    tar_mem = dataset.getmember(name)
    
    #Check if it is a file
    if(tar_mem.isfile() and name.endswith('.jpg')):
        #Build directory and class info
        im_dir = name[0:name.rfind('/')]
        im_class = im_dir[im_dir.rfind('/')+1:]
        #Add record to the dataframe
        df.loc[df.shape[0]] = [im_class, name]

In [7]:
#extract data
dataset.extractall(path='')

In [8]:
!ls -l flower_photos

total 616
drwx------ 2 270850 5000  36864 Feb 10  2016 daisy
drwx------ 2 270850 5000  53248 Feb 10  2016 dandelion
-rw-r----- 1 270850 5000 418049 Feb  9  2016 LICENSE.txt
drwx------ 2 270850 5000  36864 Feb 10  2016 roses
drwx------ 2 270850 5000  36864 Feb 10  2016 sunflowers
drwx------ 2 270850 5000  45056 Feb 10  2016 tulips


Create Training & Test Dataset

In [9]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [10]:
train_df.to_csv('flower_photos/train.csv',index=False)
test_df.to_csv('flower_photos/test.csv', index=False)

### Read training and test data

In [11]:
#Read training and test Dataframe
train_df = pd.read_csv('flower_photos/train.csv')
test_df = pd.read_csv('flower_photos/test.csv')

In [ ]:
#Check contents
train_df.sample(n=5)

In [12]:
#Get class names
class_names = train_df['class'].unique().tolist()
print('Flower classes: ', class_names)

Flower classes:  ['tulips', 'daisy', 'sunflowers', 'dandelion', 'roses']


### Build Batch generator (using ImageDataGenerator)

In [13]:
import tensorflow as tf
import numpy as np

In [14]:
#Define some parameters
img_size = 224
img_depth = 3

Function to normalize image according to Model being used

In [15]:
def normalize_data(img):
    
    #Normalize for ResNet50
    return tf.keras.applications.resnet50.preprocess_input(img)

Defime ImageDataGenerator for both Training and Test Separately

In [16]:
#Define Training Data Generator with augmentations
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                                width_shift_range=0.2,
                                                                height_shift_range=0.2,
                                                                horizontal_flip=True,
                                                                preprocessing_function=normalize_data) #Normalize the data accordingly

#Define Test Data Generator with NO augmentations
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=normalize_data) #Normalize the data accordingly

Create Data Generators objects for Training and Test

In [ ]:
train_df.head()

In [17]:
#Training (from dataframe)
train_generator = train_datagen.flow_from_dataframe(train_df, 
                                                    x_col='image_file', #File path for image
                                                    y_col='class',           #Class for the image
                                                    target_size=(img_size, img_size), #Image resize dimensions
                                                    batch_size=64)

Found 2936 validated image filenames belonging to 5 classes.


In [18]:
#Test (from dataframe)
test_generator = test_datagen.flow_from_dataframe(test_df,
                                                  x_col='image_file', #File path for image
                                                  y_col='class',           #Class for the image
                                                  target_size=(img_size, img_size), #Image resize dimensions
                                                  batch_size=64)

Found 734 validated image filenames belonging to 5 classes.


ImageDataGenerator has lot of useful features. Learn more about ImageDataGenerator at https://keras.io/preprocessing/image/

### Load pre-trained model

In [40]:
tf.keras.backend.clear_session()
model = tf.keras.applications.ResNet50(include_top=False, #Do not include classification layer for imagenet
                                       input_shape=(500,500,3),
                                       weights='imagenet')

In [41]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 500, 500, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 506, 506, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 250, 250, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 250, 250, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [21]:
model.output

<KerasTensor: shape=(None, 7, 7, 2048) dtype=float32 (created by layer 'conv5_block3_out')>

Freeze the layers in Pre-trained model

In [22]:
model.layers

In [23]:
len(model.layers)

175

In [42]:
#Set pre-trained model layers to not trainable
for layer in model.layers:
    layer.trainable = False

In [43]:
#Check if layers frozen
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 500, 500, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 506, 506, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 250, 250, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 250, 250, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [44]:
7*7*2048

100352

### Add FC layer for new classes

In [45]:
model.output

<KerasTensor: shape=(None, 16, 16, 2048) dtype=float32 (created by layer 'conv5_block3_out')>

In [46]:
#get Output layer of Pre0trained model
x1 = model.output

#Global average pool to reduce number of features and Flatten the output
x2 = tf.keras.layers.GlobalAveragePooling2D()(x1)

In [47]:
#Output shape of Global Average Pooling
x2

<KerasTensor: shape=(None, 2048) dtype=float32 (created by layer 'global_average_pooling2d')>

In [48]:
#Add output layer
prediction = tf.keras.layers.Dense(len(class_names),activation='softmax')(x2)

In [49]:
prediction

<KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'dense')>

### Building final model for Classification

In [50]:
#Using Keras Model class
final_model = tf.keras.models.Model(inputs=model.input, #Pre-trained model input as input layer
                                    outputs=prediction) #Output layer added

In [51]:
#Compile the model
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [52]:
#How does our overall model looks
final_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 500, 500, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 506, 506, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 250, 250, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 250, 250, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

### Train the model

In [34]:
#Saving the best model using model checkpoint callback
model_checkpoint=tf.keras.callbacks.ModelCheckpoint('flowers_resnet.h5', 
                                                    save_best_only=True, 
                                                    monitor='val_accuracy', 
                                                    mode='max', 
                                                    verbose=1)

In [35]:
final_model.fit(train_generator,
                epochs=5,
                steps_per_epoch= 2936//64,
                validation_data=test_generator,
                validation_steps = 734//64, 
                callbacks=[model_checkpoint])

Epoch 1/5
45/45 [==============================] - 54s 1s/step - loss: 1.1574 - accuracy: 0.5508 - val_loss: 0.4446 - val_accuracy: 0.8423

Epoch 00001: val_accuracy improved from -inf to 0.84233, saving model to flowers_resnet.h5
Epoch 2/5
45/45 [==============================] - 45s 988ms/step - loss: 0.4040 - accuracy: 0.8499 - val_loss: 0.3612 - val_accuracy: 0.8764

Epoch 00002: val_accuracy improved from 0.84233 to 0.87642, saving model to flowers_resnet.h5
Epoch 3/5
45/45 [==============================] - 44s 971ms/step - loss: 0.3325 - accuracy: 0.8842 - val_loss: 0.3111 - val_accuracy: 0.8864

Epoch 00003: val_accuracy improved from 0.87642 to 0.88636, saving model to flowers_resnet.h5
Epoch 4/5
45/45 [==============================] - 43s 954ms/step - loss: 0.2531 - accuracy: 0.9191 - val_loss: 0.3038 - val_accuracy: 0.8864

Epoch 00004: val_accuracy did not improve from 0.88636
Epoch 5/5
45/45 [==============================] - 43s 956ms/step - loss: 0.2493 - accuracy: 0.91

### Unfreeze some of Trained Layers in ResNet

In [ ]:
print(len(model.layers))

Let's unfreeze some layers at the end (which have high end features more specific to ImageNet)

In [36]:
#Unfreezing all layers after layer#
for layer in model.layers[170:]:
    layer.trainable = True 

In [37]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [38]:
#We will need to recompile the model
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
final_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [39]:
#Lets train for 10 steps
final_model.fit(train_generator, 
                epochs=10,
                initial_epoch=5,
                steps_per_epoch= 2936//64,
                validation_data=test_generator,
                validation_steps = 734//64, 
                callbacks=[model_checkpoint])

Epoch 6/10
45/45 [==============================] - 48s 997ms/step - loss: 0.3023 - accuracy: 0.8940 - val_loss: 0.5844 - val_accuracy: 0.8381

Epoch 00006: val_accuracy did not improve from 0.91051
Epoch 7/10
45/45 [==============================] - 43s 950ms/step - loss: 0.1747 - accuracy: 0.9379 - val_loss: 0.2794 - val_accuracy: 0.9148

Epoch 00007: val_accuracy improved from 0.91051 to 0.91477, saving model to flowers_resnet.h5
Epoch 8/10
45/45 [==============================] - 42s 936ms/step - loss: 0.1047 - accuracy: 0.9675 - val_loss: 0.2519 - val_accuracy: 0.9162

Epoch 00008: val_accuracy improved from 0.91477 to 0.91619, saving model to flowers_resnet.h5
Epoch 9/10
45/45 [==============================] - 42s 927ms/step - loss: 0.0791 - accuracy: 0.9781 - val_loss: 0.2208 - val_accuracy: 0.9290

Epoch 00009: val_accuracy improved from 0.91619 to 0.92898, saving model to flowers_resnet.h5
Epoch 10/10
45/45 [==============================] - 42s 926ms/step - loss: 0.0626 - ac

In [ ]:
#Lets train for 10 steps
final_model.fit(train_generator, 
                epochs=15,
                initial_epoch=10,
                steps_per_epoch= 2936//64,
                validation_data=test_generator,
                validation_steps = 734//64, 
                callbacks=[model_checkpoint])

In [ ]:
#Lets train for 10 steps
final_model.fit(train_generator, 
                epochs=20,
                initial_epoch=15,
                steps_per_epoch= 2936//64,
                validation_data=test_generator,
                validation_steps = 734//64, 
                callbacks=[model_checkpoint])

In [ ]:
final_model.save('flowers.h5')

In [ ]:
!ls -l

#### Residual block

In [ ]:
#Input to residual block
x = tf.keras.layers.Input(shape=(28,28,64,))

#First convolution layer
c1 = tf.keras.layers.Conv2D(32,kernel_size=(3,3), padding='same', activation='relu')(x)

#Second convolution layer
c2 = tf.keras.layers.Conv2D(64,kernel_size=(3,3), padding='same')(c1)

#Add operation
a1 = tf.keras.layers.Add()([c2,x])
r1 = tf.keras.layers.ReLU()(a1)

In [ ]:
x

In [ ]:
c1

In [ ]:
c2

In [ ]:
a1

In [ ]:
r1